checklist  
+ [x] plan1 OK
+ [x] plan2 OK

線が太く出すぎている問題を解決するには、モデルが線を**過度に太く予測することへのペナルティを強化**する必要があります。これは、主に**損失関数**と**ターゲットの表現**を調整することで改善できます。

以下の2つのアプローチを試してください。これらは独立した調整なので、一つずつ試すことを推奨します。

-----

## 1\. 🎯 ポジティブ重み (`pos_weight`) の微調整

線が太くなる最大の原因は、線画ピクセルに課しているペナルティ（`pos_weight`）が高すぎるためです。モデルは「線がある」と予測することで高い報酬を得ようとし、結果として線を太く広げようとします。

### ✅ 修正案：`pos_weight` の値をさらに下げる

現在採用している値（例: $5.0$）から、さらに下げて再学習してください。

  * **現在の設定（例）:** `pos_weight_tensor = torch.tensor(5.0).to(device)`
  * **修正案:** **`3.0`** や **`2.0`** 程度に値を下げて試してください。

<!-- end list -->

```python
pos_weight_value = 3.0 # 例: 5.0 から 3.0 へ変更
pos_weight_tensor = torch.tensor(pos_weight_value, dtype=torch.float).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
```

この値を下げることで、線の**濃さ**と**太さ**の両方が抑制され、背景とのコントラストが和らぎます。

-----

## 2\. 📏 L1損失の導入 (線の細さの強制)

`BCEWithLogitsLoss`はピクセルごとの分類誤差（線か背景か）を測りますが、線の**細さ**や**平均的な濃さ**を制御するには限界があります。

出力された線画 (`torch.sigmoid(pred)`) とターゲット (`line`) の間の**L1損失**（絶対誤差）をメイン損失に追加することで、モデルにピクセル値の誤差を最小限に抑えるよう強制し、過度に明るく（線が太く）なるのを防ぎます。

### ✅ 修正案：メイン損失にL1損失をミックスする

トレーニングループ内で、`BCEWithLogitsLoss`と`L1Loss`を組み合わせて使用します。

```python
# トレーニングループ内
for rough, line in loader:
    # ... (前処理はそのまま) ...
    
    pred = model(rough)
    
    # 1. メイン損失 (BCE with pos_weight)
    loss_main_bce = criterion(pred, line)
    
    # 2. L1損失 (Sigmoid後の出力とターゲットの絶対誤差)
    # L1損失を導入することで、ピクセル値がターゲットに近づくように強制する。
    loss_main_l1 = F.l1_loss(torch.sigmoid(pred), line) 
    
    # 3. メイン損失の組み合わせとエッジ損失
    # BCEとL1をミックス (例: 80% BCE, 20% L1)
    loss_main = 0.8 * loss_main_bce + 0.2 * loss_main_l1 
    
    loss_edge = edge_loss(torch.sigmoid(pred), line) 

    # 総損失 (L1損失が太さを抑制する助けになるため、エッジ重みは 1.0 で維持してもよい)
    loss = loss_main + 1.0 * loss_edge 
    
    # ... (backward 以降はそのまま) ...
```

ここで、`L1Loss`の重み（例: $0.2$）を調整することで、線の太さに対する抑制力を制御できます。

**推奨手順としては、まず `pos_weight` の値を下げて再学習し、効果が不十分であれば L1 損失の導入を検討してください。**

In [1]:
# train_unet_res_dilated.py
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.optim as optim

2025-11-17 03:19:15.056644560 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


In [2]:
# -----------------------
# Residual Block
# -----------------------
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv1 = nn.Conv2d(ch, ch, 3, padding=1)
        self.in1 = nn.InstanceNorm2d(ch)
        self.conv2 = nn.Conv2d(ch, ch, 3, padding=1)
        self.in2 = nn.InstanceNorm2d(ch)

    def forward(self, x):
        h = F.relu(self.in1(self.conv1(x)))
        h = self.in2(self.conv2(h))
        return x + h


# -----------------------
# Dilated Conv Block
# -----------------------
class DilatedConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, dilation=2):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, dilation=dilation, padding=dilation)
        self.inorm = nn.InstanceNorm2d(out_ch)

    def forward(self, x):
        return F.relu(self.inorm(self.conv(x)))


In [3]:

# -----------------------
# U-Net Generator (1ch Input / 1ch Output)
# -----------------------
class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()

        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1), 
            nn.InstanceNorm2d(64),
            nn.ReLU(True),
            ResBlock(64)
        )
        self.pool1 = nn.MaxPool2d(2)

        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(True),
            ResBlock(128)
        )
        self.pool2 = nn.MaxPool2d(2)

        self.enc3 = nn.Sequential(
            DilatedConvBlock(128, 256, dilation=2),
            ResBlock(256)
        )
        self.pool3 = nn.MaxPool2d(2)


        self.bottleneck = nn.Sequential(
        DilatedConvBlock(256, 512, dilation=4), 
        ResBlock(512),
        DilatedConvBlock(512, 512, dilation=4), 
        )

        self.up3 = nn.ConvTranspose2d(512, 256, 3, stride=2, padding=1, output_padding=1)
        self.dec3 = nn.Sequential(
        nn.Conv2d(512, 256, 3, padding=1),
        nn.InstanceNorm2d(256),
        nn.ReLU(True),
        ResBlock(256)
        )

        self.up2 = nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1)
        self.dec2 = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(True),
            ResBlock(128)
        )
        
        self.up1 = nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1)
        self.dec1 = nn.Sequential(
            DilatedConvBlock(128, 64, dilation=1),
            ResBlock(64)
        )

        self.out_conv = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool1(e1)

        e2 = self.enc2(p1)
        p2 = self.pool2(e2)

        e3 = self.enc3(p2)
        p3 = self.pool3(e3)

        b = self.bottleneck(p3)

        u3 = self.up3(b)
        d3 = self.dec3(torch.cat([u3, e3], dim=1))

        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, e2], dim=1))

        u1 = self.up1(d2)
        d1 = self.dec1(torch.cat([u1, e1], dim=1))

        return self.out_conv(d1)

In [4]:
def sobel_edges(x):
    sobel_x = torch.tensor([[-1,0,1],[-2,0,2],[-1,0,1]],
                           dtype=torch.float32, device=x.device).view(1,1,3,3)
    sobel_y = torch.tensor([[-1,-2,-1],[0,0,0],[1,2,1]],
                           dtype=torch.float32, device=x.device).view(1,1,3,3)

    g_x = F.conv2d(x, sobel_x, padding=1)
    g_y = F.conv2d(x, sobel_y, padding=1)

    return torch.sqrt(g_x**2 + g_y**2 + 1e-6)
    
def edge_loss(pred, target):
    return F.l1_loss(sobel_edges(pred), sobel_edges(target))

In [5]:
# Dataset
class SketchDataset(torch.utils.data.Dataset):
    def __init__(self, rough_dir, line_dir, transform=None):
        self.rough_files = sorted(os.listdir(rough_dir))
        self.line_files = sorted(os.listdir(line_dir))
        self.rough_dir = rough_dir
        self.line_dir = line_dir
        self.transform = transform

    def __len__(self):
        return len(self.rough_files)

    def __getitem__(self, idx):
        rough = Image.open(os.path.join(self.rough_dir, self.rough_files[idx])).convert("L")
        line  = Image.open(os.path.join(self.line_dir,  self.line_files[idx])).convert("L")

        if self.transform:
            rough = self.transform(rough)
            line  = self.transform(line)

        return rough, line

In [8]:

# ---------------------------------------------------------------------------
# II. データローディングと設定
# ---------------------------------------------------------------------------
# Dataset クラスはそのまま使用 (paired_transform 引数は不要)

# Transform の修正: Resize を 256x256 に変更
transform = transforms.Compose([
    transforms.Resize((256, 256)), # ★ 128x128 から 256x256 に変更 ★
    transforms.ToTensor(),
])

dataset = SketchDataset("dataset/train/rough", "dataset/train/line", transform)

# DataLoader: 解像度アップに伴い、VRAM節約のためバッチサイズを 2 に下げることを推奨
loader = DataLoader(dataset, batch_size=2, shuffle=True) 

device = "cuda" if torch.cuda.is_available() else "cpu"

model = UNetGenerator(in_channels=1, out_channels=1).to(device)


pos_weight_value = 3.0 
pos_weight_tensor = torch.tensor(pos_weight_value, dtype=torch.float).to(device)

# 2. criterion に Tensor 型の pos_weight を渡す
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
# オプティマイザ: 学習率を 0.0001 に下げる (暴走防止)
optimizer = optim.Adam(model.parameters(), lr=0.0001) 

os.makedirs("checkpoints", exist_ok=True)


# ---------------------------------------------------------------------------
# III. トレーニングループ
# ---------------------------------------------------------------------------
for epoch in range(50):
    total_loss = 0.0
    num_batches = 0
    model.train() 
    
    for rough, line in loader:
        rough, line = rough.to(device), line.to(device)

        line = 1.0 - line
        
        optimizer.zero_grad()
        pred = model(rough)
    
        # 1. メイン損失 (BCE with pos_weight)
        loss_main_bce = criterion(pred, line)
    
        # 2. L1損失 (Sigmoid後の出力とターゲットの絶対誤差)
        # L1損失を導入することで、ピクセル値がターゲットに近づくように強制する。
        loss_main_l1 = F.l1_loss(torch.sigmoid(pred), line) 
    
        # 3. メイン損失の組み合わせとエッジ損失
        # BCEとL1をミックス (例: 80% BCE, 20% L1)
        loss_main = 0.8 * loss_main_bce + 0.2 * loss_main_l1 
    
        loss_edge = edge_loss(torch.sigmoid(pred), line) 

        # 総損失 (L1損失が太さを抑制する助けになるため、エッジ重みは 1.0 で維持してもよい)
        loss = loss_main + 1.0 * loss_edge

        loss.backward()
        optimizer.step()
        
        # ログ改善
        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch+1}: avg_loss={avg_loss:.4f}")

    torch.save(model.state_dict(), f"checkpoints/unet_1ch_epoch{epoch+1}.pth")

Epoch 1: avg_loss=0.7899
Epoch 2: avg_loss=0.6730
Epoch 3: avg_loss=0.6409
Epoch 4: avg_loss=0.6223
Epoch 5: avg_loss=0.6099
Epoch 6: avg_loss=0.6017
Epoch 7: avg_loss=0.5959
Epoch 8: avg_loss=0.5918
Epoch 9: avg_loss=0.5862
Epoch 10: avg_loss=0.5841
Epoch 11: avg_loss=0.5804
Epoch 12: avg_loss=0.5799
Epoch 13: avg_loss=0.5774
Epoch 14: avg_loss=0.5736
Epoch 15: avg_loss=0.5719
Epoch 16: avg_loss=0.5682
Epoch 17: avg_loss=0.5658
Epoch 18: avg_loss=0.5681
Epoch 19: avg_loss=0.5686
Epoch 20: avg_loss=0.5610
Epoch 21: avg_loss=0.5551
Epoch 22: avg_loss=0.5509
Epoch 23: avg_loss=0.5469
Epoch 24: avg_loss=0.5411
Epoch 25: avg_loss=0.5401
Epoch 26: avg_loss=0.5309
Epoch 27: avg_loss=0.5263
Epoch 28: avg_loss=0.5219
Epoch 29: avg_loss=0.5181
Epoch 30: avg_loss=0.5119
Epoch 31: avg_loss=0.5044
Epoch 32: avg_loss=0.4969
Epoch 33: avg_loss=0.4902
Epoch 34: avg_loss=0.4818
Epoch 35: avg_loss=0.4727
Epoch 36: avg_loss=0.4664
Epoch 37: avg_loss=0.4553
Epoch 38: avg_loss=0.4475
Epoch 39: avg_loss=0.

In [9]:
import torch
import torchvision.transforms.functional as TF
from PIL import Image
# ... (UNetGeneratorの定義が必要) ...

device = "cuda" if torch.cuda.is_available() else "cpu"

model = UNetGenerator(in_channels=1, out_channels=1).to(device)
# チェックポイントのロード...
model.load_state_dict(torch.load("checkpoints/unet_1ch_epoch50.pth", map_location=device))
model.eval()

img = Image.open("test/rough/sample.jpg").convert("L")

# ★★★ リサイズ解像度を 256x256 に変更 ★★★
img = TF.resize(img, (256, 256)) 
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★

img = TF.to_tensor(img).unsqueeze(0).to(device)

with torch.no_grad():
    out = torch.sigmoid(model(img))  # 0〜1

out = 1.0 - out 

out = out.clamp(0,1)
out_img = TF.to_pil_image(out[0].cpu())
out_img.save("results/plan2.png")

print("done")

done


result
![result](./results/plan1.png)1
![result](./results/plan2.png)2

sample  
![sample](./test/rough/sample.jpg)